In [1]:
import geopandas as gpd

In [2]:
filepath = "qol-data/NPA_2014_meck.shp"

In [3]:
data = gpd.read_file(filepath)

In [4]:
type(data)

geopandas.geodataframe.GeoDataFrame

In [5]:
npa_data = data

In [6]:
data.crs

<Projected CRS: PROJCS["NAD83 / North Carolina (ftUS)",GEOGCS["NAD ...>
Name: NAD83 / North Carolina (ftUS)
Axis Info [cartesian]:
- [east]: Easting (US survey foot)
- [north]: Northing (US survey foot)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [7]:
data.dtypes

ACRES        float64
NPA            int64
geometry    geometry
dtype: object

In [8]:
#change crs to match coordinates of rezoning petition centroids 
#created new df
data_2 = data.to_crs({'init': 'epsg:4326'})

C:\Users\gshre\Anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [9]:
# Dependencies
import os
import requests
import json
import pandas as pd
import types

# from geojson_utils import centroid # does not like decimals

#def save_to_csv(df, filename):
#    path = os.path.dirname(__file__)
#    df_csv = f"{path}/../resources/{filename}"
#    df.to_csv(df_csv, index=False)


def getCentralPoint(feature):
    lat = 0.0
    lng = 0.0
    record_count = 0
    for coordinates in feature['geometry']['coordinates']:
        for coordinate in coordinates:
            if not all(isinstance(x, list) for x in coordinate):
                lng += coordinate[0]
                lat += coordinate[1]
                record_count+= 1
            else:
                for coordinate2 in coordinate:
                    lng += coordinate2[0]
                    lat += coordinate2[1]        
                    record_count+= 1        

    centroid = {}
    centroid['lng'] = lng / record_count
    centroid['lat'] = lat / record_count

    return centroid

url = 'https://opendata.arcgis.com/datasets/19c8803ab1214b21859127d463034520_23.geojson'

geo_data = requests.get(url).json()

rezoning_history = []
for feature in geo_data['features']:
    rezoning_request = feature["properties"]
    rezoning_request["centroid"] = getCentralPoint(feature)
    rezoning_history.append(rezoning_request)


rezoning_history_df = pd.DataFrame(rezoning_history)
#save_to_csv(rezoning_history_df, "rezoning_history.csv")
rezoning_history_df.head()

# Print the json
#print(geo_data)

,OBJECTID,Petition,Petitioner,FromZone,ToZone,Type,SPA,Overlay,INNOV,Acres,...,TOTUNITS,OFFICE,RETAIL,HOTELRMS,PARKING,OPENSP,ACRESITE,SHAPESTArea,SHAPESTLength,centroid
0,565704,2008-069,Charlotte Douglas Airport,I-2(CD),I-2,NC,no,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,6.048340e+06,17135.428780,"{'lng': -80.9473629555874, 'lat': 35.192261963..."
1,565705,2014-004,Charlotte-Mecklenburg Planning Department,I-2,TOD-M,NC,no,none,NO,2.40,...,NaN,NaN,NaN,NaN,NaN,None,None,1.165708e+05,1386.958932,"{'lng': -80.79401695812554, 'lat': 35.25197931..."
2,565706,2008-052,"Novant Health, Inc.",R-3,B-1(CD),CD,no,None,None,81.75,...,NaN,NaN,NaN,NaN,NaN,None,None,8.079010e+05,5226.629743,"{'lng': -80.64636364121559, 'lat': 35.22259419..."
3,565707,2014-049,"SBBH, LLC",CC,MUDD-O,CD,no,none,NO,1.53,...,NaN,NaN,NaN,NaN,NaN,None,None,5.781933e+04,1024.520620,"{'lng': -80.82910095839671, 'lat': 35.15347976..."
4,565708,1992-069,None,"B-1, O-1",B-2(CD),None,no,None,None,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,4.128143e+05,3377.246656,"{'lng': -80.83262067298986, 'lat': 35.14814504..."


In [10]:
npa_data.head()

,ACRES,NPA,geometry
0,411.111570,2,"POLYGON ((1467904.864 537018.911, 1467870.041 ..."
1,1155.559009,3,"POLYGON ((1446321.249 531915.105, 1446200.404 ..."
2,332.138420,4,"POLYGON ((1461919.601 518486.370, 1462036.397 ..."
3,167.098401,5,"POLYGON ((1436326.378 552625.615, 1436297.715 ..."
4,403.117080,6,"POLYGON ((1439771.024 545112.988, 1439762.230 ..."


In [11]:
from shapely.geometry import Point, Polygon

In [12]:
#Loop through every row in rezoning history df to see if centroid is located 
#within polygon in each row of data_2 df
matches_dict = {}
for index, request_row in rezoning_history_df.iterrows():
    p1 = Point(request_row["centroid"]["lng"],request_row["centroid"]["lat"])
    #print(p1)
    for index, npa_row in data_2.iterrows():
        npa_poly = npa_row["geometry"]
        #print(npa_poly)
        if npa_poly.contains(p1):
            matches_dict[request_row["Petition"]]=npa_row["NPA"]
    
       

In [13]:
print(matches_dict)

{'2008-069': 122, '2014-004': 21, '2008-052': 228, '2014-049': 358, '1992-069': 213, '2008-059': 331, '1991-002(C)': 122, '2007-054': 230, '1988-031': 178, '1988-069': 387, '2007-044': 180, '1988-040': 384, '2006-127': 345, '2005-074': 51, '2017-040': 339, '2019-102': 116, '2017-125': 174, '1989-051(C)': 60, '1996-078': 114, '2002-085': 231, '2002-143': 392, '2008-147': 113, '2004-082': 364, '2007-107': 341, '2006-031': 341, '2006-051': 155, '2015-014': 359, '2000-053': 48, '1993-082C': 290, '1988-062': 348, '1994-048': 341, '2019-080': 390, '2002-078': 113, '1988-042': 247, '1993-069D': 381, '2004-062': 30, '1987-036(C)': 48, '1998-024(C)': 360, '1985-055': 315, '2018-169': 339, '1992-043': 182, '1987-010': 3, '1987-055': 357, '2006-134': 177, '1999-076': 328, '1985-079': 157, '2008-022': 230, '1996-048(C)': 306, '1993-095A': 382, '2013-012': 160, '2018-149': 347, '1986-077': 140, '1988-044': 197, '2018-115': 209, '1998-027(C)': 250, '1990-041': 389, '2006-035': 347, '2015-087': 284, 

In [14]:
#create new df
rezoning_history_df_2 = rezoning_history_df

In [15]:
#map dictionary of NPA's that match rezoning request centroids onto rezoning_history_df_2 to create new column "NPA_Located"
rezoning_history_df_2["NPA_Located"]=rezoning_history_df_2["Petition"].map(matches_dict)

In [16]:
rezoning_history_df_2.head()

,OBJECTID,Petition,Petitioner,FromZone,ToZone,Type,SPA,Overlay,INNOV,Acres,...,OFFICE,RETAIL,HOTELRMS,PARKING,OPENSP,ACRESITE,SHAPESTArea,SHAPESTLength,centroid,NPA_Located
0,565704,2008-069,Charlotte Douglas Airport,I-2(CD),I-2,NC,no,None,None,NaN,...,NaN,NaN,NaN,NaN,None,None,6.048340e+06,17135.428780,"{'lng': -80.9473629555874, 'lat': 35.192261963...",122.0
1,565705,2014-004,Charlotte-Mecklenburg Planning Department,I-2,TOD-M,NC,no,none,NO,2.40,...,NaN,NaN,NaN,NaN,None,None,1.165708e+05,1386.958932,"{'lng': -80.79401695812554, 'lat': 35.25197931...",21.0
2,565706,2008-052,"Novant Health, Inc.",R-3,B-1(CD),CD,no,None,None,81.75,...,NaN,NaN,NaN,NaN,None,None,8.079010e+05,5226.629743,"{'lng': -80.64636364121559, 'lat': 35.22259419...",228.0
3,565707,2014-049,"SBBH, LLC",CC,MUDD-O,CD,no,none,NO,1.53,...,NaN,NaN,NaN,NaN,None,None,5.781933e+04,1024.520620,"{'lng': -80.82910095839671, 'lat': 35.15347976...",358.0
4,565708,1992-069,None,"B-1, O-1",B-2(CD),None,no,None,None,NaN,...,NaN,NaN,NaN,NaN,None,None,4.128143e+05,3377.246656,"{'lng': -80.83262067298986, 'lat': 35.14814504...",213.0


In [17]:
#drop NA values for 'NPA_Located' column (only 1 NA value)
rezoning_history_df_2.dropna(subset=['NPA_Located'],inplace=True)

In [18]:
rezoning_history_df_2.dtypes

OBJECTID           int64
Petition          object
Petitioner        object
FromZone          object
ToZone            object
Type              object
SPA               object
Overlay           object
INNOV             object
Acres            float64
PetAcres         float64
Water             object
Sewer             object
CityLimit         object
ZoningMap         object
ODZ               object
CCW               object
ServiceArea       object
LuseStaff         object
UrbanDesign       object
RezoneStaff       object
Consistent        object
ProtestPet        object
Received          object
Approved          object
Decision          object
Hyperlink         object
Corrective        object
Propluse          object
AdminAppr         object
AdminApprD        object
AdminApprP        object
AdminApprC        object
SFLOTS           float64
THUNITS          float64
CONDOS            object
APT              float64
TOTUNITS         float64
OFFICE           float64
RETAIL           float64


In [19]:
#change "NPA_Located" from float to integer
rezoning_history_df_2[["NPA_Located"]] = rezoning_history_df_2[["NPA_Located"]].astype(int)

In [20]:
rezoning_history_df_2.head()


,OBJECTID,Petition,Petitioner,FromZone,ToZone,Type,SPA,Overlay,INNOV,Acres,...,OFFICE,RETAIL,HOTELRMS,PARKING,OPENSP,ACRESITE,SHAPESTArea,SHAPESTLength,centroid,NPA_Located
0,565704,2008-069,Charlotte Douglas Airport,I-2(CD),I-2,NC,no,None,None,NaN,...,NaN,NaN,NaN,NaN,None,None,6.048340e+06,17135.428780,"{'lng': -80.9473629555874, 'lat': 35.192261963...",122
1,565705,2014-004,Charlotte-Mecklenburg Planning Department,I-2,TOD-M,NC,no,none,NO,2.40,...,NaN,NaN,NaN,NaN,None,None,1.165708e+05,1386.958932,"{'lng': -80.79401695812554, 'lat': 35.25197931...",21
2,565706,2008-052,"Novant Health, Inc.",R-3,B-1(CD),CD,no,None,None,81.75,...,NaN,NaN,NaN,NaN,None,None,8.079010e+05,5226.629743,"{'lng': -80.64636364121559, 'lat': 35.22259419...",228
3,565707,2014-049,"SBBH, LLC",CC,MUDD-O,CD,no,none,NO,1.53,...,NaN,NaN,NaN,NaN,None,None,5.781933e+04,1024.520620,"{'lng': -80.82910095839671, 'lat': 35.15347976...",358
4,565708,1992-069,None,"B-1, O-1",B-2(CD),None,no,None,None,NaN,...,NaN,NaN,NaN,NaN,None,None,4.128143e+05,3377.246656,"{'lng': -80.83262067298986, 'lat': 35.14814504...",213


In [21]:
# Merge data_2 df with rezoning_history_df_2 
merged_epsg_4326_df = pd.merge(data_2,rezoning_history_df_2,left_on='NPA',right_on="NPA_Located")

#Rename columns
merged_epsg_4326_df.rename(columns={'ACRES': 'npa_acres', 'geometry': 'npa_geometry',}, inplace=True)

In [22]:
type(merged_epsg_4326_df)

geopandas.geodataframe.GeoDataFrame

In [23]:
#convert dataframe to csv
merged_epsg_4326_df.to_csv("matched_npa_requests.csv")

In [36]:
npa_count_df = merged_epsg_4326_df.groupby("NPA").count()['npa_acres'].reset_index()

In [42]:
npa_count_df.sort_values(by='npa_acres', ascending=False).head(20)

,NPA,npa_acres
105,116,436
1,3,231
301,339,202
329,367,112
305,343,97
157,174,91
350,388,75
19,22,66
147,163,61
65,73,60


In [49]:
npa_decision_count_df = merged_epsg_4326_df.groupby(["NPA", 'Decision']).count()['npa_acres'].reset_index()

In [54]:
year_df = merged_epsg_4326_df['Petition'].str[:4]
merged_epsg_4326_df['petition_year'] = year_df

In [56]:
npa_year_decicion_df = merged_epsg_4326_df.groupby(['petition_year', "NPA", 'Decision']).count()['npa_acres'].reset_index()

In [59]:
npa_year_decicion_df.sort_values(by='npa_acres', ascending=False).head(40)

,petition_year,NPA,Decision,npa_acres
2674,2019,116,App,361
2617,2018,339,App,132
1757,2008,122,App,36
86,1989,3,App,27
1517,2006,3,Wd,20
1609,2006,347,App,19
1434,2005,73,App,18
1411,2005,3,App,16
1626,2007,3,App,14
2235,2015,22,App,13


In [48]:
npa_decision_count_df.sort_values(by='npa_acres', ascending=False).head(20)

,NPA,Decision,npa_acres
150,116,App,419
434,339,App,196
2,3,App,193
442,343,App,92
482,367,App,70
230,174,App,70
515,388,App,69
32,22,App,63
97,73,App,56
215,163,App,53


In [24]:
merged_epsg_4326_df.isna().sum()

npa_acres           0
NPA                 0
npa_geometry        0
OBJECTID            0
Petition            0
Petitioner       2811
FromZone          494
ToZone              1
Type             2786
SPA                23
Overlay          4096
INNOV            4141
Acres            3057
PetAcres         2957
Water            2990
Sewer            2987
CityLimit        2956
ZoningMap        2973
ODZ              4798
CCW              3570
ServiceArea      2922
LuseStaff        2929
UrbanDesign      2925
RezoneStaff      3046
Consistent       4849
ProtestPet       5345
Received         2932
Approved          320
Decision          499
Hyperlink        2720
Corrective       5867
Propluse         5206
AdminAppr        6095
AdminApprD       6096
AdminApprP       6099
AdminApprC       6089
SFLOTS           6199
THUNITS          6199
CONDOS           6203
APT              6188
TOTUNITS         6195
OFFICE           6194
RETAIL           6192
HOTELRMS         6200
PARKING          6202
OPENSP    